In [1]:
# 初始化参数，获取列名
from sklearn.ensemble import IsolationForest
from sklearn import svm
import pandas as pd
import numpy as np
import sys
import csv

params = {}
params['n_estimators'] = 100
params['max_samples'] ='auto'
params['contamination'] = 0.1
params['max_features'] = 1.0

params['nu'] = 0.001
params['gamma']='auto'
params['kernel'] ='poly'

params['path'] = 'data/train/B01.csv'
params['opath'] ='dataclean/B01.csv'
try:
    with open(params['path'],'r') as f:
        reader = csv.reader(f)
        head_row=next(reader)
        data_attribute = []
    for item in head_row:
        data_attribute.append(item)
    print(data_attribute)
except Exception as e:
    print(e)

['DE_time', 'FE_time', 'BA_time', 'RPM']


In [2]:
# dataclean
tn = pd.read_csv(params['path']) 
tn.dropna(inplace=True)
train = np.array(tn)
train_x = np.array(train)
print(train_x)
# clf = IsolationForest(n_estimators=params['n_estimators'], 
#                       max_samples=params['max_samples'], 
#                       contamination=params['contamination'], 
#                       max_features=params['max_features'], 
#                       bootstrap=False, n_jobs=1, random_state=None, 
#                       verbose=0).fit(train_x)
clf = svm.OneClassSVM(nu=params['nu'],
              kernel=params['kernel'],
              gamma=params['gamma']).fit(train_x)
pred = clf.predict(train_x)
print (pred.size)

[[ 1.91511018e-01 -3.69818182e-02 -8.20842730e-03  1.79600000e+03]
 [ 1.50577365e-01  4.49945455e-02 -1.55316320e-02  1.79600000e+03]
 [-3.68727745e-02 -8.62909091e-03  3.82255193e-03  1.79600000e+03]
 ...
 [ 2.42353214e-01 -6.98545455e-03  7.06970920e-02  1.79600000e+03]
 [ 3.24870259e-03  1.06836364e-02  7.12604154e-02  1.79600000e+03]
 [-6.22126547e-02 -1.01700000e-01 -2.03198813e-02  1.79600000e+03]]
14708


In [7]:
# 保存清洗后数据
df = pd.DataFrame(pd.read_csv(params['path']))[0:pred.size]
df['pred']=pred
df2 = df[-df.pred.isin([-1])]
df2 = df2.drop(['pred'],axis=1)
data_out = df2.iloc[:,:].values
print(int(df2.size/4))
csvfile2 = open(params['opath'],'w',newline='')
writer = csv.writer(csvfile2)
writer.writerow(data_attribute)   #存属性
m=len(data_out)
for i in range(m):
    writer.writerow(data_out[i])

14708


In [38]:
# 特征提取
import pandas as pd
import numpy as np
import os
from scipy import stats,signal,fftpack
import math
from pywt import wavedec
import traceback
import sys
columns_list=['time_mean','time_std','time_max','time_min','time_rms','time_ptp','time_median','time_iqr','time_pr','time_skew','time_kurtosis','time_var','time_amp',                'time_smr','time_wavefactor','time_peakfactor','time_pulse','time_margin','freq_mean','freq_std','freq_max','freq_min','freq_rms','freq_median',               'freq_iqr','freq_pr','freq_f2','freq_f3','freq_f4','freq_f5','freq_f6','freq_f7','freq_f8','ener_cA5','ener_cD1','ener_cD2','ener_cD3','ener_cD4',        'ener_cD5','ratio_cA5','ratio_cD1','ratio_cD2','ratio_cD3','ratio_cD4','ratio_cD5']
columns_list1 = [a + '_DE' for a in columns_list]
columns_list2 = [a + '_FE' for a in columns_list]
# columns_list3 = [a + '_BA' for a in columns_list]
# columns_list_final = columns_list1 + columns_list2+ columns_list3
columns_list_final = columns_list1 + columns_list2
params = {}
params['len_piece']=50 #窗口长度
windowlen=params['len_piece']


def feature_get(filepath,windowlen):
        df_data = pd.DataFrame(pd.read_csv(filepath))
#         dfs = df_data.loc[:,['DE_time','FE_time','BA_time']]
        dfs = df_data.loc[:,['DE_time','FE_time']]
        features_list=[]
        print(len(dfs))
        for i in range (0,len(dfs),windowlen):
            if(int((len(dfs)-i)/100)>=1): #舍去少量数据
                df=dfs[i:i+params['len_piece']]
                feature_list = []
    #             print(df)
                for i in df.columns:
                    #----------  time-domain feature,18
                    #依次为均值，标准差，最大值，最小值，均方根，峰峰值，中位数，四分位差，百分位差，偏度，峰度，方差，整流平均值，方根幅值，波形因子，峰值因子，脉冲值，裕度
                    df_line = df[i]
    #                 print(df_line)
                    time_mean = df_line.mean()
                    time_std = df_line.std()
                    time_max = df_line.max()
                    time_min = df_line.min()
                    time_rms = np.sqrt(np.square(df_line).mean())
                    time_ptp = time_max-time_min 
                    time_median = np.median(df_line)
                    time_iqr = np.percentile(df_line,75)-np.percentile(df_line,25)
                    time_pr = np.percentile(df_line,90)-np.percentile(df_line,10)
                    time_skew = stats.skew(df_line)
                    time_kurtosis = stats.kurtosis(df_line)
                    time_var = np.var(df_line)
                    time_amp = np.abs(df_line).mean()
                    time_smr = np.square(np.sqrt(np.abs(df_line)).mean())
                    #下面四个特征需要注意分母为0或接近0问题，可能会发生报错
                    time_wavefactor = time_rms/time_amp
                    time_peakfactor = time_max/time_rms
                    time_pulse = time_max/time_amp
                    time_margin = time_max/time_smr
                    #----------  freq-domain feature,15
                    #采样频率25600Hz
                    df_fftline = fftpack.fft(df[i])
                    freq_fftline = fftpack.fftfreq(len(df[i]),1/25600)
                    df_fftline = abs(df_fftline[freq_fftline>=0])
                    freq_fftline = freq_fftline[freq_fftline>=0]
                    #基本特征,依次为均值，标准差，最大值，最小值，均方根，中位数，四分位差，百分位差
                    freq_mean = df_fftline.mean()
                    freq_std = df_fftline.std()
                    freq_max = df_fftline.max()
                    freq_min = df_fftline.min()
                    freq_rms = np.sqrt(np.square(df_fftline).mean())
                    freq_median = np.median(df_fftline)
                    freq_iqr = np.percentile(df_fftline,75)-np.percentile(df_fftline,25)
                    freq_pr = np.percentile(df_fftline,90)-np.percentile(df_fftline,10)
                    #f2 f3 f4反映频谱集中程度
                    freq_f2 = np.square((df_fftline-freq_mean)).sum()/(len(df_fftline)-1)
                    freq_f3 = pow((df_fftline-freq_mean),3).sum()/(len(df_fftline)*pow(freq_f2,1.5))
                    freq_f4 = pow((df_fftline-freq_mean),4).sum()/(len(df_fftline)*pow(freq_f2,2))
                    #f5 f6 f7 f8反映主频带位置
                    freq_f5 = np.multiply(freq_fftline,df_fftline).sum()/df_fftline.sum()
                    freq_f6 = np.sqrt(np.multiply(np.square(freq_fftline),df_fftline).sum())/df_fftline.sum()
                    freq_f7 = np.sqrt(np.multiply(pow(freq_fftline,4),df_fftline).sum())/np.multiply(np.square(freq_fftline),df_fftline).sum()
                    freq_f8 = np.multiply(np.square(freq_fftline),df_fftline).sum()/np.sqrt(np.multiply(pow(freq_fftline,4),df_fftline).sum()*df_fftline.sum())
                    #----------  timefreq-domain feature,12
                    #5级小波变换，最后输出6个能量特征和其归一化能量特征
                    cA5, cD5, cD4, cD3, cD2, cD1 = wavedec(df[i], 'db10', level=5)
                    ener_cA5 = np.square(cA5).sum()
                    ener_cD5 = np.square(cD5).sum()
                    ener_cD4 = np.square(cD4).sum()
                    ener_cD3 = np.square(cD3).sum()
                    ener_cD2 = np.square(cD2).sum()
                    ener_cD1 = np.square(cD1).sum()
                    ener = ener_cA5 + ener_cD1 + ener_cD2 + ener_cD3 + ener_cD4 + ener_cD5
                    ratio_cA5 = ener_cA5/ener
                    ratio_cD5 = ener_cD5/ener
                    ratio_cD4 = ener_cD4/ener
                    ratio_cD3 = ener_cD3/ener
                    ratio_cD2 = ener_cD2/ener
                    ratio_cD1 = ener_cD1/ener
                    feature_list.extend([time_mean,time_std,time_max,time_min,time_rms,time_ptp,time_median,time_iqr,time_pr,time_skew,time_kurtosis,time_var,time_amp,
                                     time_smr,time_wavefactor,time_peakfactor,time_pulse,time_margin,freq_mean,freq_std,freq_max,freq_min,freq_rms,freq_median,
                                     freq_iqr,freq_pr,freq_f2,freq_f3,freq_f4,freq_f5,freq_f6,freq_f7,freq_f8,ener_cA5,ener_cD1,ener_cD2,ener_cD3,ener_cD4,ener_cD5,
                                     ratio_cA5,ratio_cD1,ratio_cD2,ratio_cD3,ratio_cD4,ratio_cD5])
                features_list.append(feature_list)
        return features_list

# 约定normal(NORMAL), ball(B), outer race(OR), inner race(IR)的预测输出标签为0, 1, 2, 3。
# NORMAL:0
for i in range(1,3):
    params['data_path'] = 'data/train/NORMAL0'+str(i)+'.csv'
    params['opath'] ='datafeature/NORMAL0'+str(i)+'_feature.csv'
    fea = feature_get(params['data_path'],windowlen)
    result = pd.DataFrame(fea,columns = columns_list_final)
    result['label']=0
    result.to_csv(params['opath'],index=False,header=True)

# B:1    
for i in range(1,7):
    params['data_path'] = 'data/train/B0'+str(i)+'.csv'
    params['opath'] ='datafeature/B0'+str(i)+'_feature.csv'
    fea = feature_get(params['data_path'],windowlen)
    result = pd.DataFrame(fea,columns = columns_list_final)
    result['label']=1
    result.to_csv(params['opath'],index=False,header=True)
    
# OR:2    
for i in range(1,15):
    if(i<=9):
        params['data_path'] = 'data/train/OR0'+str(i)+'.csv'
        params['opath'] ='datafeature/OR0'+str(i)+'_feature.csv'
    else:
        params['data_path'] = 'data/train/OR'+str(i)+'.csv'
        params['opath'] ='datafeature/OR'+str(i)+'_feature.csv'        
    fea = feature_get(params['data_path'],windowlen)
    result = pd.DataFrame(fea,columns = columns_list_final)
    result['label']=2
    result.to_csv(params['opath'],index=False,header=True)
    
# IR:3
for i in range(1,7):
    params['data_path'] = 'data/train/IR0'+str(i)+'.csv'
    params['opath'] ='datafeature/IR0'+str(i)+'_feature.csv'
    fea = feature_get(params['data_path'],windowlen)
    result = pd.DataFrame(fea,columns = columns_list_final)
    result['label']=3
    result.to_csv(params['opath'],index=False,header=True)
    

# TEST:-1
for i in range(1,15):
    if(i<=9):
        params['data_path'] = 'data/test/TEST0'+str(i)+'.csv'
        params['opath'] ='testfeature/TEST0'+str(i)+'_feature.csv'
    else:
        params['data_path'] = 'data/test/TEST'+str(i)+'.csv'
        params['opath'] ='testfeature/TEST'+str(i)+'_feature.csv'        
    fea = feature_get(params['data_path'],windowlen)
    result = pd.DataFrame(fea,columns = columns_list_final)
    result['label']=-1
    result.to_csv(params['opath'],index=False,header=True)

print("done")

182953


C:\Users\yjw98\Anaconda3\envs\tensorflow\lib\site-packages\pywt\_multilevel.py:45: UserWarning: Level value of 5 is too high: all coefficients will experience boundary effects.
  "boundary effects.").format(level))


362927
14708
14621
14638
14569
14656
14604
7336
7319
7336
7310
7302
7345
7310
7310
7345
7319
7328
7319
7319
7345
14551
14621
14656
14638
14621
14586
14586
14656
14621
7293
7336
7336
7362
363797
7336
7310
7284
14621
14656
14638
done


In [39]:
# 合并训练集
import os
import pandas as pd
filePath='datafeature'
outputfile='train.csv'
fileList = os.listdir(filePath)
# print(fileList)
# print(len(fileList))
path = os.path.join(filePath, fileList[0])
df = pd.read_csv(path)
df.to_csv(outputfile,index=False, header=True)
for i in range (1,len(fileList)):
    path = os.path.join(filePath, fileList[i])
#     print(path)
    df = pd.read_csv(path)
    df.to_csv(outputfile, mode='a', index=False, header=False)
print("Done.")

Done.


In [40]:
# LGBM分类
import lightgbm as lgb
import joblib
def train():
    # 模型初始化，设置random_state保证可复现性，便于观察优化
    train_data = pd.read_csv('train.csv')
    train_data_y = train_data['label']
    # 除去标签的所有列就是特征
    train_data_x = train_data.drop(['label'], axis=1)
    model_lgb_default = lgb.LGBMClassifier(learning_rate=0.03,random_state=None)
    # 模型训练
    model_lgb_default.fit(train_data_x, train_data_y)
    joblib.dump(model_lgb_default, 'model/lightgbm_model.model')
    print("Done.")

train()

Done.


In [42]:
# knn分类
import lightgbm as lgb
import joblib
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

def train():
    # 模型初始化，设置random_state保证可复现性，便于观察优化
    train_data = pd.read_csv('train.csv')
    train_data_y = train_data['label']
    # 除去标签的所有列就是特征
    train_data_x = train_data.drop(['label'], axis=1)
    train_x, valid_x, train_y, valid_y = train_test_split(train_data_x, train_data_y, test_size=0.3, random_state=0)
    # 传入到knn进行训练，得到模型
    classifier = KNeighborsClassifier()
    param_dict = {"n_neighbors":[1,3,5,7,9]}
   #加入网格搜索和交叉验证
    classifier = GridSearchCV(classifier,param_grid=param_dict,cv=4)
    classifier.fit(train_x,train_y)
 
    y_predict = classifier.predict(valid_x)
    print("最佳参数 ：",classifier.best_params_)
    print("最佳结果：",classifier.best_score_)
    print("最佳预估器：",classifier.best_estimator_)
    print("交叉验证结果",classifier.cv_results_)
    print("准确度为：", classifier.score(valid_x,valid_y))
    joblib.dump(classifier, 'model/knn.model')
    print("Done.")


train()

最佳参数 ： {'n_neighbors': 7}
最佳结果： 0.9106550658597297
最佳预估器： KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=7, p=2,
                     weights='uniform')
交叉验证结果 {'mean_fit_time': array([0.17730254, 0.17129713, 0.17153877, 0.17227316, 0.17377406]), 'std_fit_time': array([0.00310815, 0.00130852, 0.00157492, 0.00042172, 0.003342  ]), 'mean_score_time': array([0.17852104, 0.20121932, 0.21692711, 0.23238713, 0.24963623]), 'std_score_time': array([0.00612883, 0.00658459, 0.00631842, 0.00435995, 0.00884237]), 'param_n_neighbors': masked_array(data=[1, 3, 5, 7, 9],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'n_neighbors': 1}, {'n_neighbors': 3}, {'n_neighbors': 5}, {'n_neighbors': 7}, {'n_neighbors': 9}], 'split0_test_score': array([0.89746263, 0.90858533, 0.91067084, 0.9092805 , 0.91032325]), 'split1_test_score': array([0.89016336

In [17]:
# 超参数搜索（BUGing）
import lightgbm as lgb
import joblib
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

def train():
    # 模型初始化，设置random_state保证可复现性，便于观察优化
    train_data = pd.read_csv('train.csv')
    train_data_y = train_data['label']
    # 除去标签的所有列就是特征
    train_data_x = train_data.drop(['label'], axis=1)
    train_x, valid_x, train_y, valid_y = train_test_split(train_data_x, train_data_y, test_size=0.333, random_state=0)
    train = lgb.Dataset(train_x, train_y)
    valid = lgb.Dataset(valid_x, valid_y, reference=train)
    parameters = {
              'max_depth': [15, 20, 25, 30, 35],
              'learning_rate': [0.05, 0.1, 0.15],
              'feature_fraction': [0.6, 0.7, 0.8, 0.9, 0.95],
              'bagging_fraction': [0.6, 0.7, 0.8, 0.9, 0.95],
              'bagging_freq': [2, 4, 5, 6, 8],
              'lambda_l1': [0, 0.1, 0.4, 0.5, 0.6],
              'lambda_l2': [0, 10, 15, 35, 40],
              'cat_smooth': [1, 10, 15, 20, 35]
    }
    gbm = lgb.LGBMClassifier(boosting_type='gbdt',
                         objective = 'binary',
                         metric = 'auc',
                         verbose = 0,
                         learning_rate = 0.01,
                         num_leaves = 35,
                         feature_fraction=0.8,
                         bagging_fraction= 0.9,
                         bagging_freq= 8,
                         lambda_l1= 0.6,
                         lambda_l2= 0)
    gsearch = GridSearchCV(gbm, param_grid=parameters, scoring='accuracy', cv=3)
    gsearch.fit(train_x, train_y)
    print("Best score: %0.3f" % gsearch.best_score_)
    print("Best parameters set:")
    best_parameters = gsearch.best_estimator_.get_params()
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))
#     joblib.dump(model_lgb_default, 'model/lightgbm_model.model')
    print("Done.")

train()

C:\Users\yjw98\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
lightgbm.basic.LightGBMError: Multiclass objective and metrics don't match

  FitFailedWarning)


KeyboardInterrupt: 

In [44]:
# Try to use tensorflow DNN
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd
import joblib

def input_fn(features, labels, training=True, batch_size=256):
    """An input function for training or evaluating"""
    # 将输入转换为数据集。
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # 如果在训练模式下混淆并重复数据。
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)


# 模型初始化，设置random_state保证可复现性，便于观察优化
train_data = pd.read_csv('train.csv')
train_data_y = train_data['label']
# 除去标签的所有列就是特征
train_data_x = train_data.drop(['label'], axis=1)
train_x, test_x, train_y, test_y = train_test_split(train_data_x, train_data_y, test_size=0.25, random_state=0)
my_feature_columns = []
for key in train_x.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # 隐层所含结点数量分别为 30 和 10.
    hidden_units=[30, 10],
    # 模型必须从三个类别中做出选择。
    n_classes=4)
classifier.train(
    input_fn=lambda: input_fn(train_x, train_y, training=True),
    steps=5000)




INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\yjw98\\AppData\\Local\\Temp\\tmpr7d8sfnq', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001AE89B5BDD8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:C

In [45]:
# 评估train_test_split中划分的测试集
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(test_x, test_y, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-03-25T21:15:23Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\yjw98\AppData\Local\Temp\tmpr7d8sfnq\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2020-03-25-21:15:24
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.8452555, average_loss = 0.644

In [49]:
# 由tf模型生成预测
expected = ['NORMAL', 'B', 'OR','IR']
test1=pd.read_csv('testfeature/TEST01_feature.csv')
test2=pd.read_csv('testfeature/TEST02_feature.csv')
test3=pd.read_csv('testfeature/TEST03_feature.csv')
test4=pd.read_csv('testfeature/TEST04_feature.csv')
test5=pd.read_csv('testfeature/TEST05_feature.csv')
test6=pd.read_csv('testfeature/TEST06_feature.csv')
test7=pd.read_csv('testfeature/TEST07_feature.csv')
test8=pd.read_csv('testfeature/TEST08_feature.csv')
test9=pd.read_csv('testfeature/TEST09_feature.csv')
test10=pd.read_csv('testfeature/TEST10_feature.csv')
test11=pd.read_csv('testfeature/TEST11_feature.csv')
test12=pd.read_csv('testfeature/TEST12_feature.csv')
test13=pd.read_csv('testfeature/TEST13_feature.csv')
test14=pd.read_csv('testfeature/TEST14_feature.csv')
def input_fn(features, batch_size=256):
    """An input function for prediction."""
    # 将输入转换为无标签数据集。
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)
i=1
testsets=[test1,test2,test3,test4,test5,test6,test7,test8,test9,test10,test11,test12,test13,test14]
names=[]
predlists=[]
ans=[]
for test_i in testsets:
    test_name = 'test_' + str(i)
    i+=1
    test_j=test_i.drop(['label'], axis=1)
    predict_x=test_i
    predictions = classifier.predict(
        input_fn=lambda: input_fn(predict_x))
    y_pred_binary=[]
    for pred in predictions:
        y_pred_binary.append(pred['class_ids'][0])
    predlist=[y_pred_binary.count(0), y_pred_binary.count(1),y_pred_binary.count(2),y_pred_binary.count(3)]
    pred=predlist.index(max(predlist))
    names.append(test_name)
    ans.append(pred)
    predlists.append(predlist)
    
for name,predlist,pred in zip(names,predlists,ans):
    print(name,predlist,pred)
    
# for pred_dict, expec in zip(predictions, expected):
#     class_id = pred_dict['class_ids'][0]
#     probability = pred_dict['probabilities'][class_id]

#     print('Prediction is "{}" ({:.1f}%), expected "{}"'.format(
#         expected[class_id], 100 * probability, expec))

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\yjw98\AppData\Local\Temp\tmpr7d8sfnq\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\yjw98\AppData\Local\Temp\tmpr7d8sfnq\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\yjw98\AppData\Local\Temp\tmpr7d8sfnq\model.ckpt-5000
INFO:tensorflow:Running local_i

In [43]:
# 预测结果
import lightgbm as lgb
import joblib
import pandas as pd

def judge(input_pred):
    return_pred = list(np.zeros(len(input_pred)))
    for i in range(0, len(input_pred)):
        if (input_pred[i][0] > 0.5):
            return_pred[i] = 0
        elif(input_pred[i][1]>0.5):
            return_pred[i] = 1
        elif(input_pred[i][2]>0.5):
            return_pred[i] = 2
        elif(input_pred[i][3]>0.5):
            return_pred[i] = 3
    return return_pred
    
def score(predlist ,pred):
    # score = [0.3×f1score(class1) + 0.3×f1score(class2) + 0.3×f1score(class3) + 0.1×f1score(class0)]*100
    pass

def test_lightgbm():
    # 加载模型
#     model = joblib.load('model/lightgbm_model.model')
    model = joblib.load('model/knn.model')
    test1=pd.read_csv('testfeature/TEST01_feature.csv')
    test2=pd.read_csv('testfeature/TEST02_feature.csv')
    test3=pd.read_csv('testfeature/TEST03_feature.csv')
    test4=pd.read_csv('testfeature/TEST04_feature.csv')
    test5=pd.read_csv('testfeature/TEST05_feature.csv')
    test6=pd.read_csv('testfeature/TEST06_feature.csv')
    test7=pd.read_csv('testfeature/TEST07_feature.csv')
    test8=pd.read_csv('testfeature/TEST08_feature.csv')
    test9=pd.read_csv('testfeature/TEST09_feature.csv')
    test10=pd.read_csv('testfeature/TEST10_feature.csv')
    test11=pd.read_csv('testfeature/TEST11_feature.csv')
    test12=pd.read_csv('testfeature/TEST12_feature.csv')
    test13=pd.read_csv('testfeature/TEST13_feature.csv')
    test14=pd.read_csv('testfeature/TEST14_feature.csv')
    i=1
    for test_i in [test1,test2,test3,test4,test5,test6,test7,test8,test9,test10,test11,test12,test13,test14]:
        test_name = 'test_' + str(i)
        i+=1
        print('The result of test is:')
        test_j=test_i.drop(['label'], axis=1)
        y_pred = model.predict_proba(test_j)
        y_pred_binary = judge(y_pred)
        predlist=[y_pred_binary.count(0), y_pred_binary.count(1),y_pred_binary.count(2),y_pred_binary.count(3)]
        pred=predlist.index(max(predlist))
        print(test_name,predlist ,pred)


        
test_lightgbm()

The result of test is:
test_1 [3, 286, 1, 0] 1
The result of test is:
test_2 [22, 260, 6, 4] 1
The result of test is:
test_3 [36, 198, 28, 29] 1
The result of test is:
test_4 [3, 0, 85, 56] 2
The result of test is:
test_5 [19, 50, 68, 8] 2
The result of test is:
test_6 [1, 1, 98, 45] 2
The result of test is:
test_7 [12, 61, 52, 21] 1
The result of test is:
test_8 [7272, 2, 0, 0] 0
The result of test is:
test_9 [18, 41, 69, 17] 2
The result of test is:
test_10 [19, 44, 78, 4] 2
The result of test is:
test_11 [11, 4, 117, 12] 2
The result of test is:
test_12 [29, 17, 83, 162] 3
The result of test is:
test_13 [0, 0, 8, 284] 3
The result of test is:
test_14 [48, 219, 20, 4] 1
